[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/00_Quickstart.ipynb)


In [1]:
import os
import sys
from pathlib import Path
import subprocess

def _ensure_repo():
    workspace = os.environ.get('GITHUB_WORKSPACE')
    if workspace:
        repo_root = Path(workspace).resolve()
        src_path = repo_root / 'src'
        if src_path.exists():
            sys.path.insert(0, str(src_path))
            return repo_root
    repo_root = Path().resolve()
    src_path = repo_root / 'src'
    if src_path.exists():
        sys.path.insert(0, str(src_path))
        return repo_root
    repo_dir = Path('IINTS-SDK')
    if not repo_dir.exists():
        subprocess.check_call(['git', 'clone', 'https://github.com/python35/IINTS-SDK.git'])
    repo_root = repo_dir.resolve()
    os.chdir(repo_root)
    src_path = repo_root / 'src'
    sys.path.insert(0, str(src_path))
    return repo_root

_ensure_repo()


PosixPath('/home/runner/work/IINTS-SDK/IINTS-SDK')

# IINTS-AF SDK Quickstart

This notebook gets you from zero to a full simulation run, including audit + PDF report.


## Setup

If you are running locally in the repo:

```bash
python3 -m pip install -e .
```


In [2]:
import json
from iints.presets import load_presets

import iints
from iints.core.algorithms.pid_controller import PIDController

presets = load_presets()
preset = next(p for p in presets if p["name"] == "baseline_t1d")

scenario = preset["scenario"]
patient_config = preset["patient_config"]
duration = preset["duration_minutes"]
time_step = preset["time_step_minutes"]

outputs = iints.run_simulation(
    algorithm=PIDController,
    scenario=scenario,
    patient_config=patient_config,
    duration_minutes=duration,
    time_step=time_step,
    seed=42,
    output_dir="results/quickstart",
)

outputs.keys()


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


dict_keys(['results', 'safety_report', 'baseline_comparison', 'results_csv', 'audit', 'baseline_files', 'report_pdf'])

## Quick Metrics Summary


In [3]:
from iints.metrics import calculate_full_metrics

metrics = calculate_full_metrics(outputs["results"]["glucose_actual_mgdl"], duration_hours=duration/60)
metrics


ClinicalMetricsResult(tir_70_180=np.float64(38.46153846153847), tir_70_140=np.float64(38.46153846153847), tir_70_110=np.float64(23.076923076923077), tir_below_70=np.float64(61.53846153846154), tir_below_54=np.float64(53.84615384615385), tir_above_180=np.float64(0.0), tir_above_250=np.float64(0.0), cv=np.float64(69.01737235969001), sd=40.77795488873483, gmi=np.float64(4.723279943280238), mean_glucose=59.08360966890625, median_glucose=47.31090125000001, hi=np.float64(33.53416567445627), lbgi=np.float64(4.166873449414023), hbgi=np.float64(0.013564730924593123), readings_per_day=13.0, data_coverage=4.513888888888888)

## Quickstart PDF Export


In [4]:
quickstart_pdf = iints.generate_quickstart_report(
    outputs["results"],
    "results/quickstart/quickstart_report.pdf",
    outputs["safety_report"],
)
quickstart_pdf


'results/quickstart/quickstart_report.pdf'

## Maker Faire Demo PDF


In [5]:
demo_pdf = iints.generate_demo_report(
    outputs["results"],
    "results/quickstart/demo_report.pdf",
    outputs["safety_report"],
)
demo_pdf


'results/quickstart/demo_report.pdf'

In [6]:
outputs["results"].head()


,time_minutes,glucose_actual_mgdl,glucose_to_algo_mgdl,delivered_insulin_units,algo_recommended_insulin_units,sensor_status,pump_status,pump_reason,basal_insulin_units,bolus_insulin_units,...,uncertainty,fallback_triggered,safety_level,safety_actions,safety_reason,safety_triggered,supervisor_latency_ms,human_intervention,human_intervention_note,algorithm_why_log
0,0,140.000000,140.000000,3.0,3.2,ok,ok,approved,0,3.2,...,0.0,False,warning,WINDOW_CAP_EXCEEDED: 60min cap 3.00U reached,WINDOW_CAP_EXCEEDED: 60min cap 3.00U reached,True,0.023614,False,,[{'reason': 'Glucose error from target (120 mg...
1,5,118.666667,118.666667,0.0,0.0,ok,ok,approved,0,0.0,...,0.0,False,safe,NEGATIVE_TREND_LIMIT: Glucose dropping at -4.2...,NEGATIVE_TREND_LIMIT: Glucose dropping at -4.2...,True,0.013716,False,,[{'reason': 'Glucose error from target (120 mg...
2,10,100.200000,100.200000,0.0,0.0,ok,ok,approved,0,0.0,...,0.0,False,safe,NEGATIVE_TREND_LIMIT: Glucose dropping at -3.9...,NEGATIVE_TREND_LIMIT: Glucose dropping at -3.9...,True,0.010619,False,,[{'reason': 'Glucose error from target (120 mg...
3,15,84.170000,84.170000,0.0,0.0,ok,ok,approved,0,0.0,...,0.0,False,safe,NEGATIVE_TREND_LIMIT: Glucose dropping at -3.4...,NEGATIVE_TREND_LIMIT: Glucose dropping at -3.4...,True,0.009107,False,,[{'reason': 'Glucose error from target (120 mg...
4,20,70.211167,70.211167,0.0,0.0,ok,ok,approved,0,0.0,...,0.0,False,safe,NEGATIVE_TREND_LIMIT: Glucose dropping at -3.0...,NEGATIVE_TREND_LIMIT: Glucose dropping at -3.0...,True,0.009217,False,,[{'reason': 'Glucose error from target (120 mg...


In [7]:
outputs["safety_report"]


{'total_violations': 13,
 'violation_breakdown': {'safe': 4,
  'warning': 1,
  'critical': 1,
  'emergency': 7},
 'bolus_interventions_count': 1,
 'emergency_mode_active': True,
 'current_iob': 2.4000000000000004,
 'recent_violations': [{'level': 'emergency',
   'message': 'HYPO_CUTOFF: Glucose below safety cutoff; EMERGENCY_STOP: Severe hypoglycemia detected',
   'glucose': 29.532126153125006,
   'time': 40.0,
   'original_proposed_insulin': 0.0,
   'approved_insulin': 0},
  {'level': 'emergency',
   'message': 'HYPO_CUTOFF: Glucose below safety cutoff; EMERGENCY_STOP: Severe hypoglycemia detected',
   'glucose': 22.102307230156256,
   'time': 45.0,
   'original_proposed_insulin': 0.0,
   'approved_insulin': 0},
  {'level': 'emergency',
   'message': 'HYPO_CUTOFF: Glucose below safety cutoff; EMERGENCY_STOP: Severe hypoglycemia detected',
   'glucose': 20.0,
   'time': 50.0,
   'original_proposed_insulin': 0.0,
   'approved_insulin': 0},
  {'level': 'emergency',
   'message': 'HYPO_CU

In [8]:
outputs.get("report_pdf")


'results/quickstart/clinical_report.pdf'